Projet de Données à la Connaissance
======

## Introduction

## Choix de l'algorithme de clustering
DBSCAN 
raison du choix :
- Non-flat geometry, 
- uneven cluster sizes
- Très scalable
- Nombre de clusters inconnus (vs K-means)

In [1]:
import pandas as pd # Ouverture des données
import numpy as np 
import gensim # Pour lire word2vec
import math
import sys
from scipy.sparse import dok_matrix, csc_matrix

/Users/quentin/anaconda/lib/python3.6/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory: 'sysctl'
  stacklevel=stacklevel + 1):


# Ouverture des données

In [2]:
df_posts=pd.read_csv("data/reddit_posts.csv",sep=";", names=['Id','UserName','SubReddit','Title','Body'], header=None)
print("Posts loaded into dataframe")
df_comments=pd.read_csv("data/reddit_comments.csv",sep=";", names=['Id','UserName','SubReddit','Comment'], header=None)
print("Comments loaded into dataframe")

Posts loaded into dataframe
Comments loaded into dataframe


In [3]:
print(len(df_posts))
print(df_posts.head())
print(len(df_comments))
print(df_comments.head())

4468114
   Id        UserName  SubReddit  \
0   1           SRASC      Honda   
1   3         JRabone   Patriots   
2   4         adamorn  AskReddit   
3   5        JakeM917  doctorwho   
4  10  DataScienceInc    pystats   

                                               Title       Body  
0                                     The new S2000?        NaN  
1      I made a video for SB LI supporting our Pats!        NaN  
2  Trump supporters - Do you feel that he is doin...  [removed]  
3                         The 13th Doctor we deserve        NaN  
4                        Introduction to Correlation        NaN  
35561917
   Id        UserName            SubReddit  \
0   1        crash787        pokemontrades   
1   2    Cuntosaurous                  Amd   
2   3         dwmills  KitchenConfidential   
3   4  GodsGiftInBeef          LiverpoolFC   
4   5       Smeeks765      ecigclassifieds   

                                             Comment  
0                        Regigigas and

In [19]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("data/GoogleNews-vectors-negative300.bin.gz", binary=True)

# Première approche : Clustering des SubReddits par NLP

In [ ]:
import nltk

stop_words = set(stopwords.words('english'))

# Deuxième approche : Clustering des SubReddits par Utilisateurs

In [88]:
df_posts_dropped = df_posts.drop(labels=['Title', 'Body'], axis=1)
df_comments_dropped = df_comments.drop(labels='Comment', axis=1)

users = df_posts_dropped["UserName"].append(df_comments_dropped["UserName"])
users.drop_duplicates(inplace=True)
nb_users = len(users)

subreddits = df_posts_dropped["SubReddit"].append(df_comments_dropped["SubReddit"])
subreddits.drop_duplicates(inplace=True)
nb_subreddits = len(subreddits)

all_data = df_posts_dropped.append(df_comments_dropped)

print(len(df_posts_dropped), nb_users, nb_subreddits)

4468114 3330748 94288


In [89]:
grouped_users = all_data.groupby("UserName")["Id"].count()
grouped_users_removed = list(grouped_users[grouped_users > 10].index)
print(grouped_users_removed[:50], len(grouped_users_removed))

grouped_subreddits = all_data.groupby("SubReddit")["Id"].count()
grouped_subreddits_removed = list(grouped_subreddits[grouped_subreddits > 10].index)
print(grouped_subreddits_removed[:50], len(grouped_subreddits_removed))

['----------_--------', '--------__--------', '--------cool--------', '-------_-----', '------________', '-----BroAway-----', '-----McSqeezie-----', '-----_------_---', '-----_____-----_____', '-----fuck-----', '-----iMartijn-----', '----MXE----', '----Nomad----', '----Val----', '----__-__----', '----_____----', '----josh', '----root', '---Blue---', '---CMFinley---', '---David---', '---E', '---Earth---', '---JustMe---', '---Kai---', '---Lucifer---', '---R', '---S-U-C-C---', '---TheFierceDeity---', '---YNWA---', '---_---_---_---7', '---_--__-_-----', '---_-___', '---fantastic---', '---lol', '---o0o---', '---sniff---', '--23---', '--444--', '--ACAB--', '--AJ--', '--ANUSTART-', '--ATCQ--', '--AXIOM--', '--Aeo--', '--Arrow--', '--BMO--', '--Beetlejuice--', '--BlueHat--', '--CAT--'] 695616
['01246', '07Scape', '098f6bcd4621d373', '0ad', '0x3642', '1000thworldproblems', '100DaysofKeto', '100in28', '100movies365days', '100pushups', '100sets', '100sexiest', '100yearclub', '100yearsago', '101Wi

In [91]:
import random

users_id, subreddits_id = {}, {}
user_item_matrix = dok_matrix((len(grouped_users_removed), len(grouped_subreddits_removed)), dtype=int)

grouped_comments = all_data.groupby(["UserName","SubReddit"])["Id"].count().reset_index()
print("toto")
grouped_comments = grouped_comments.loc[(grouped_comments["UserName"].isin(grouped_users_removed)) & (grouped_comments["SubReddit"].isin(grouped_subreddits_removed))]

print(type(grouped_comments))
print(grouped_comments)
c = 0
for index,item in grouped_comments.iterrows():
    c += 1
    username = item["UserName"]
    subreddit = item["SubReddit"]
    number = item["Id"]
    try:
        i = users_id[username]
    except:
        i = len(users_id)
        users_id[username] = i
    try:
        j = subreddits_id[subreddit]
    except:
        j = len(subreddits_id)
        subreddits_id[subreddit] = j
    user_item_matrix[i, j] = int(math.ceil(math.log(1 + number)))
    if c%100000 == 0:
        print(c, sys.getsizeof(user_item_matrix), sys.getsizeof(subreddits_id))

print("Finished")

toto
<class 'pandas.core.frame.DataFrame'>
                      UserName             SubReddit  Id
13         ----------_--------       leagueoflegends   9
14         ----------_--------                   wow   3
20          --------__--------              Winnipeg  72
21        --------cool--------               Fitness  10
22        --------cool--------              ultimate  18
24               -------_-----             AskReddit   1
25               -------_-----  Competitiveoverwatch  12
26               -------_-----            Futurology   2
27               -------_-----       GlobalOffensive  26
28               -------_-----                 Jokes   1
29               -------_-----          OurPresident   2
30               -------_-----             Overwatch  15
31               -------_-----   OverwatchUniversity   3
32               -------_-----                 funny   2
33               -------_-----                gaming   1
34               -------_-----               

In [92]:
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

N_CLUSTERS = 1000
N_COMPONENTS = 300
BATCH_SIZE = 40000

print(len(users_id))
print(user_item_matrix.shape)

X = csr_matrix(user_item_matrix).transpose()
print("Transposed")

tsvd = TruncatedSVD(n_components = N_COMPONENTS)
X = tsvd.fit_transform(X)
print("Truncated")

del db
db = MiniBatchKMeans(n_clusters=N_CLUSTERS, batch_size=BATCH_SIZE).fit_predict(X)
print("Finished")

695567
(695616, 32607)
Transposed
Truncated
Finished


In [93]:
for n in range(N_CLUSTERS):
    l = []
    for subreddit in subreddits_id:
        cluster = db[subreddits_id[subreddit]]
        if cluster == n:
            l.append(subreddit)
    print("Cluster n°{}, size: {}".format(n, len(l)))
    print(l)
    print("\n")

Cluster n°0, size: 256
['ToME4', 'Augusta', 'COROLLA', 'Taxidermy', 'extrawoods', 'MobileWallpaper', 'liquor', 'tortoise', 'Charlottesville', 'LibraryofBabel', 'atlanticcity', 'TheOffspring', 'Metaphysics', 'NOTSONEWREDDITS', 'obscuresubreddits', 'gout', 'NoRules', 'knots', 'Reggaeton', 'thedoors', 'LaserDisc', 'energydrinks', 'HBOGameofThrones', 'fatherted', 'electroswing', 'coloradohikers', 'SOCOM', 'dnbproduction', 'asianbros', 'SoundSystem', 'BarBattlestations', 'candy', 'ski', '360video', 'csun', 'jewelers', 'RedditAndChill', 'Trichsters', 'PimpYourMomForKarma', 'Oahu', 'coopplay', 'cheesemaking', 'oracle', 'wholesomegreentext', 'vjing', 'newengland', 'musicfestivals', 'ConfusingGravity', 'Elvis', 'funfacts', 'UnnecessaryCensorship', 'Fosterparents', 'GetOutOfBed', 'ps3homebrew', 'audiodrama', 'Broadchurch', 'Flagstaff', 'Danger5', 'princeton', 'johnoliver', 'bladerunner', 'ns2', 'TheKillers', 'BucksCountyPA', 'PostCollapse', 'drphil', 'lalaland', 'emulators', 'Anger', 'cosplaypro

Cluster n°22, size: 22
['YGOSales', 'incremental_games', 'londonontario', 'GameTheorists', 'TheBrewery', 'dvdcollection', 'productivity', 'birding', 'ptcgo', 'singularity', 'ThreshMains', 'learnmachinelearning', 'de_EDV', 'duolingo', 'GenderCritical', 'baduk', 'brokehugs', 'AskDrugNerds', 'VintageApple', 'Elsanna', 'MathHelp', 'SeriousConversation']


Cluster n°23, size: 23
['GTAV_Cruises', 'TheFootballManagers', 'footballtactics', 'ShadyBeaver', 'gta5', 'Bell_Whiff', 'GtaCarSpawns', 'olympiquedemarseille', 'PortsmouthFC', 'gato20', 'gtafriends', 'rangersfc', 'SheffieldUnited', 'internacional', 'SevillaFC', 'LaLiga', 'AlgerianFootball', 'AustrianSports', 'XFootball', 'effzeh', 'bluebirds', 'ChampionsFantasy', '1FCNuernberg']


Cluster n°24, size: 26
['devils', 'Predators', 'stlouisblues', 'BostonBruins', 'leafs', 'EdmontonOilers', 'fantasyhockey', 'rangers', 'nhl', 'winnipegjets', 'OttawaSenators', 'sabres', 'caps', 'Flyers', 'NewYorkIslanders', 'hockeyplayers', 'SanJoseSharks', 'Dalla


Cluster n°36, size: 15
['Lettering', 'writers', 'malepolish', 'logophilia', 'mypartneristrans', '52book', 'OldManDog', 'Libraries', 'Mindfulness', 'wolves', 'pbsideachannel', 'bookshelf', 'VirginiaBeach', 'rescuedogs', 'wordplay']


Cluster n°37, size: 2
['worldofpvp', 'STPeach']


Cluster n°38, size: 61
['DogsStandingUp', 'RPGStuck', 'RPGStuck_C4', 'StammersStuck', 'kaiju', 'seventhworldproblems', 'Shittypokestops', 'panicatthedisco', 'whatwouldyoudoif', 'EVEX', 'RoughRomanMemes', 'aztrosist', 'YayorNay', 'HreWeStand', 'DubtrackOcularRP', 'Waluigi', 'FakeContext', 'Starset', 'httyd', 'DailyInkblot', 'weirdwikihow', 'ShittyAlbumArtPorn', 'farts', 'random_thought', 'Soundclown', 'DowntonAbbey', 'PocketPlanes', 'bannedfromcpisland', 'comeonandslam', 'YiffPetPlay', 'RPGStuck_C4S10_2', 'montypython', 'MyInternetDiary', 'CPPSme', 'inanimateinsanity', 'Hiveswap', 'suberbowl', 'The_Keith', 'quizzyBot', 'moodkip', 'textventures', 'blackbutler', 'duckgame', 'bepis', 'Avoid5Test', 'MarmiteBadge

Cluster n°46, size: 26
['renoise', 'vectorart', 'pico8', 'SkullJuice', 'swinburne', 'battleshops', 'Student', 'new_websites', 'SundayPhantomForces', 'Photoshop_WTF', 'badmovieideas', 'oddlyunsatisfying', 'AdiposeAmigos', 'BrilliantLightPower', 'Anu', 'MelbEnts', 'melbournemusic', 'servicedesign', 'rmit', 'Monash', 'Sarawak', 'AppleMusicAlbums', 'LaTrobe', 'murdoch', 'reptiliandude', 'smallfarmer']


Cluster n°47, size: 4
['timetolegalize', 'StonerPhilosophy', 'thetreesnetwork', 'CSGO_Skins']


Cluster n°48, size: 28
['yaoi', 'StarlightStage', 'CodeGeass', 'araragigirls', 'overlord', 'Nisekoi', 'Mobpsycho100', 'KamenRider', 'AnimeThemes', 'Clannad', 'KimiNoNaWa', 'toarumajutsunoindex', 'xxanime', 'GakkouGurashi', 'AnimeSketch', 'Crunchyroll', 'YoujoSenki', 'Keijo', 'Dakimakuras', 'OneTrueBiribiri', 'YourWaifuIsTrash', 'HibikeEuphonium', 'animelegwear', 'ReLIFE', 'AquaSama', 'AnimeShortFilms', 'seiyuu', 'watchinganime']


Cluster n°49, size: 24
['UWMadison', 'BigEast', 'bullcity', 'UDFly

Cluster n°63, size: 8
['gtavcustoms', 'PSNFriends', 'PS4Banners', 'gravityrush', 'gtacartel', 'littlebigplanet', 'PSW', 'LosSantosAutoClubPS4']


Cluster n°64, size: 148
['FindLingerie', 'postcolonialism', '360Realtors', 'smokerslounge', 'femininasmr', 'PokeTubeLP', 'LeagueOfDerp', 'ENLTPAStars', 'IsraeliMossad', 'ThylacineScience', 'MarbleGame', 'HersheyBroadcaster', 'AntyyysPapercrafts', 'Papercraftprojects', 'fullmovierequest', 'GREhelp', 'Eliteantal_testsite', 'motograndprix', 'BitcoinJobs', 'Islamophobia', 'Jameskii', 'benhoward', 'RegularFeatures', 'EuropeanEscorts', 'webm', 'KentuckyWildcats', 'Peppers', 'battlegroundeurope', 'teamdd', 'SpaceNinjasPlsIgnore', 'Traeger', 'LutonTown', 'Essay_Writing_Service', 'Urbandead', 'WorldRallyCross', 'PPeperomioides', 'RamonaSplinas', 'ludobots', 'LoFiPop', 'ChoppedandScrewed', 'CheesyMemes', 'calireggae', 'RevengeUHC', 'Bonn', 'Grumpamon', 'findnewmetal', 'Strawpolls', 'bermuda', 'StrawHatCharacterBios', 'aesthetics', 'DentonClassifieds', 

...

Cluster n°939, size: 21
['victoria2', 'transgendercirclejerk', 'SocialistRA', 'TROLLXCOMMUNISM', 'Socialism_101', 'COMPLETEANARCHY', 'communism', 'ChapoTrapHouse', 'ShitRedditSays', 'Negareddit', 'badphilosophy', 'militant', 'LeftWithoutEdge', 'communism101', 'ANI_COMMUNISM', 'forwardsfromhitler', 'FULLDISCOURSE', 'DebateAnarchism', 'Anarchy101', 'DebateCommunism', 'ABoringDystopia']


Cluster n°940, size: 134
['unofficial2b2t', 'ArcaneAdventuresTrade', 'invaderzim', 'googlesheets', 'Minesweeper', 'empyrion', 'VLC', 'HillClimbRacing', 'HalfBloodHangout', 'mindcrackcirclejerk', 'allthemods', 'Verticalwallpapers', 'DaytonaBeach', 'Ravenfield', '6b6t', 'WorseEveryLoop', 'MCBuildSchool', 'Rayman', '8b8tnet', 'thrive', 'projectozone2', 'Decksplash', 'Tay_Tweets', 'MetalGearAntiNuclear', 'brass', 'repost', 'Hexagorio', 'RunningWithDogs', 'Emojerk', 'Angory_Tom', 'puppers', 'yokaitrade', 'minecraftseeds', 'nytimes', 'botania', 'superhot', 'overcast', 'AndroidHelp', 'Eddsworld', 'reddithelp', 

Cluster n°942, size: 9
['ftlgame', 'falloutsettlements', 'HaloStory', 'Parahumans', 'CharacterRant', '40kOrkScience', 'Vermintide', 'ManyATrueNerd', '40krpg']


Cluster n°943, size: 26
['standingout', 'MiaMalkova', 'Mia_khalifa', 'POVPornVids', 'Cumontits', 'JadaStevens', 'CelebCumSluts', 'DrunkGirls', 'gape', 'FFNBPS', 'AnimeBooty', 'GirlsPlayingSports', 'AlphaBetaGammaTheta', 'Connie_Carter', 'Brawesome', 'MiaSollis', 'Blondeass', 'HotGirls', 'JordanCarver', 'Lucy_Vixen', 'SexInFrontOfOthers', 'StrugglePorn', 'Underbun', 'blondehairblueeyes', 'boltedonlips', 'topless']


Cluster n°944, size: 72
['Malmalloy', 'petplay', 'CarliKanePorn', 'Androgynoushotties', 'CuckoldXxX', 'amateur_shemales', 'OnOffDudes', 'rough_sex_gifs', 'CEI', 'uphisbutt', 'Panties_Showing', 'redditamateurvids', 'PrincessIsobelGW', 'Sexyvelma', 'scent_of_women_feet', 'wetspot', 'PedoHatersAnonymous', 'AlishaWhite', 'ThickandBBWJeans', 'londonandrews', 'cumonher', 'SideLips', 'Bad_ass_girlfriends', 'RealGirlJO', 'se

Cluster n°960, size: 30
['exchangeserver', 'Veeam', 'virtualization', 'ITdept', 'sysadminjobs', 'redhat', 'radarr', 'wireless', 'macsysadmin', 'zfs', 'cybersecurity', 'unRAID', 'VOIP', 'Citrix', 'SQLServer', 'cablegore', 'mikrotik', 'Juniper', 'WindowsServer', 'ansible', 'ccnp', 'sharepoint', 'fortinet', 'paloaltonetworks', 'labtech', 'meraki', 'CentOS', 'mysql', 'ITManagers', 'ipv6']


Cluster n°961, size: 14
['IHateSportsball', 'HeKnowsQuantumPhysics', 'niceguysDiscussion', 'TrekRP', 'FacebookCringe', 'neckbeardstories', 'IncelDiscussions', 'ShittySeduction', 'dontyouknowwhoiam', 'femaleincels', 'Irony', 'SRSFunny', 'shitpostbot5000', 'thelounge']


Cluster n°962, size: 27
['Our_Politics', 'usanews', 'conservatives', 'InternationalNews', 'FreePolDiscussion', 'The_DonaldUnleashed', 'USNEWS', 'PoliticalVideos', 'WomenForTrump', 'FreeSpeech', 'calexit', 'AmericanPolitics', 'EndlessWar', 'antiwar', 'censorship', 'open_news', 'governmentoppression', 'worldevents', 'TheRecordCorrected', 'm

Cluster n°976, size: 8
['raisedbyborderlines', 'lossprevention', 'legal', 'Adoption', 'hoarding', 'SouthBend', 'myterribleneighbors', 'AusLegal']


Cluster n°977, size: 185
['ArtPorn', 'grammar', 'StackAdvice', 'Albuquerque', 'Anticonsumption', 'Brooklyn', 'sanantonio', 'AskMenOver30', 'tdi', 'spicy', 'DestructionPorn', 'freelance', 'Pennsylvania', 'Utah', 'britishcolumbia', 'florida', 'vermont', 'AskAnthropology', 'spiders', 'IWantOut', 'mbti', 'IndoorGarden', 'VietNam', 'Omaha', 'WWIIplanes', 'stopsmoking', 'dresdenfiles', 'latterdaysaints', 'Buttcoin', 'OSU', 'IWantToLearn', 'digitalnomad', 'WoT', 'SWORDS', 'Hamilton', 'TalesFromThePizzaGuy', 'Maine', 'fakealbumcovers', 'roguelikes', 'biology', 'AskALiberal', 'Ohio', 'needadvice', 'Lightbulb', 'bizarrebuildings', 'Dogtraining', 'TheFacebookDelusion', 'aliens', 'tulsa', 'CorporateFacepalm', 'newhampshire', 'puzzles', 'datascience', 'cheatatmathhomework', 'askmath', 'OldNews', 'mead', 'NorthKoreaNews', 'Stoicism', 'Vue', 'socialanxiet

Cluster n°989, size: 6
['blackdesertonline', 'GiftofGames', 'italy', 'steam_giveaway', 'FUTMobile', 'RandomKindness']


Cluster n°990, size: 37
['Complextro', 'GreekLife', 'FemraMeta', 'HuaweiP9', 'Resume', 'shitty_confessions', 'tested', 'wholesometragedy', 'MysteryMedia', 'thesprawl', 'saxophone', 'frontierfios', 'MyRetardCousin', 'telekinesis', 'StackGunHeroes', 'CalgaryGaming', 'Nightshift', 'wegmansmasterrace', 'robinhobb', 'GnuPG', 'Askashittyparent', 'Inception', 'thedarkmountain', 'BioshockInfinite', 'mtgstorm', 'menandunderwear', 'ypsi', 'gaidhlig', 'dokha', 'CandyMakers', 'Doppleganger', 'LasCruces', 'INeedToTalkToSomeone', 'banjolele', 'PotsGG', 'GnuCash', 'AskIndia']


Cluster n°991, size: 40
['Mustang', 'Wrangler', 'metalworking', 'CherokeeXJ', 'Toyota', 'MechanicAdvice', 'Stance', 'AwesomeCarMods', 'HotWheels', 'Honda', 'battlewagon', 'Trucks', 'fordranger', 'techsupportmacgyver', 'projectcar', 'Cartalk', 'GrandCherokee', 'FiestaST', 'Trucking', 'Diesel', 'overlanding', '

# Conclusion